# Testing Notebook

I am going to run some functionality tests for componants of the final programme.

## Fetch Book Info. from ISBN

Creating a class that initializes with an ISBN and creates a Book object with details for the Library GUI.

LibraryThing API key ( I didn't end up using this):

Token Name: LibraryScanner

Token: 02b372f198a5ec0ee4e0dab3f2dc99f2

In [11]:
import requests
from IPython.display import display, Image

class Book:
    def __init__(self, isbn, title=None, author=None, published=None, cover=None):
        self.isbn = isbn
        self.title = title
        self.author = author
        self.published_date = published
        self.cover_image_url = cover

    def get_data(self):
        ''' Uses the 'isbn' class variable to query the Google Books catalogue

        DEPENDENCIES
        -------------
        requests
        IPython.display (for notebook)

        NOTES
        ------
        Google Books API documentation:
        https://developers.google.com/books/docs/v1/using

        When trying decide which credentials to create for Google Cloud JSON API
        (https://console.cloud.google.com/apis/credentials?project=pythonlibraryscanner), 
        the recommendation is:
            This API doesn't require that you create credentials. 
            You're already good to go!
        '''
        api_url = f'https://www.googleapis.com/books/v1/volumes?q=isbn:{self.isbn}'

        try:
            response = requests.get(api_url)
            response.raise_for_status()
            data = response.json()

            # Extract relevant information from the API response
            if 'items' in data and data['items']:
                book_info = data['items'][0]['volumeInfo']
                self.title = book_info.get('title', 'N/A')
                self.author = ', '.join(book_info.get('authors', ['N/A']))
                self.published_date = book_info.get('publishedDate', 'N/A')

                # Choose the largest available cover image size
                # https://stackoverflow.com/questions/10721886/how-to-get-the-extra-large-cover-image-from-google-book-api
                image_links = book_info.get('imageLinks', {})
                sizes = ['extraLarge', 'large', 'medium', 'small', 'thumbnail', 'smallThumbnail']
                finding_biggest = True
                i = 0
                while finding_biggest:
                    size = sizes[i]
                    if size in image_links:
                        self.cover_image_url = image_links[size]
                        # The api will just give you the biggest image it has up to the number you set. 
                        # You can also set the height if you need to using e.g., '&fife=h900' 
                        # and both with e.g., '&fife=w800-h900'
                        self.cover_image_url = self.cover_image_url +'&fife=w600'
                        finding_biggest = False
                    i += 1
                    if i > len(sizes):  # cover image not found
                        self.cover_image_url = None
                        finding_biggest = False

        except requests.exceptions.RequestException as e:
            print(f"Error retrieving data from Google Books API: {e}")

    def display_info(self):
        print(f"Title: {self.title}")
        print(f"Author: {self.author}")
        print(f"Published Date: {self.published_date}")

        if self.cover_image_url is not None:
            display(Image(url=self.cover_image_url, width=200))
        print(f"Cover Image URL: {self.cover_image_url}")





In [2]:
# Example usage:
isbn_to_query = '9780141030012'
book_instance = Book(isbn_to_query)
book_instance.get_data()
book_instance.display_info()

Title: Things I Want My Daughters to Know
Author: Elizabeth Noble
Published Date: 2008-09-04


Cover Image URL: http://books.google.com/books/content?id=e7iqjRF-dygC&printsec=frontcover&img=1&zoom=1&source=gbs_api&fife=w600


## Get ISBN from Barcode

## File Architecture

In [15]:
import csv
import os

class Library:
    def __init__(self, library_file=None):
        
        if library_file is None:
            library_file = 'library.csv'
        self.library_file = library_file

        self.books = {}  # empty shelves
        self.header = ['isbn', 'title', 'author', 'published', 'cover']  # the basic header

        if not os.path.exists(library_file):
            # Create an empty CSV file
            with open(library_file, 'w', newline=''):
                pass
        self.load_books(library_file)

    def load_books(self,library_file=None):
        ''' can be used to merge multiple library_files'''
        if library_file is None:
             library_file = self.library_file

        with open(library_file, 'r', newline='') as file:
            reader = csv.DictReader(file) 
            for row in reader:
                self.books[row['isbn']] = Book(row['isbn'], title=row['title'], author=row['author']\
                                               , published=row['published'], cover=row['cover'])
            
            #new_header = row.keys()
            # will need to resolve header merging when extra columns are optionally added
            
            # will need to tack on extra column options when added.

    def save_library(self, save_as=False):
        ''' overwrites the library csv file (self.library_file; default='Library.csv')

        DEPENDENCIES
        -------------
        csv
        '''

        if save_as:  # True for strings
             file_path = save_as
        else:
             file_path = self.library_file
        with open(file_path, 'w', newline='') as file:
                    writer = csv.DictWriter(file, fieldnames=self.header)
                    writer.writeheader()
                    for book in self.books:
                        print(book)
                        #writer.writerow(book)

    def add_book(self, isbn, save=True):
        book_object = Book(isbn)
        book_object.get_data()
        
        self.books[isbn] = (book_object.isbn, book_object.title, book_object.author\
                            , book_object.published_date, book_object.cover_image_url)
        
        if save:
            self.save_library()

    def add_books(self,isbn_book_list):
        ''' adds all the books in the provided list to the library file. 
         Saves once at the end of the process

         INPUT
         ------
         isbn_book_list:    type: list
                            subtype: string (10 or 13 digit real, whole numbers)
                            contains: a list of ISBN numbers as strings
        '''

        for book_isbn in isbn_book_list:
            self.add_book(book_isbn, save=False)
        self.save_library()

In [16]:
# Example usage:
isbn_to_query = '9780141030012'
new_library = Library()
new_library.add_book(isbn_to_query)


9780141030012


## GUI

In [23]:
import ipywidgets as widgets

class GalleryGUI:
    def __init__(self):
        self.books = []  # List to store Book objects
        self.list_widget = widgets.Select(options=[], description="Books:", layout={'height': '200px'})
        self.add_button = widgets.Button(description="Add Book")
        self.remove_button = widgets.Button(description="Remove Book")
        self.output_text = widgets.Output()

        # Register the event handlers for the buttons
        self.add_button.on_click(self.add_book)
        self.remove_button.on_click(self.remove_book)

        # Display the widgets
        display(widgets.VBox([self.list_widget,
                              widgets.HBox([self.add_button, self.remove_button]),
                              self.output_text]))

    def add_book(self, _):
        isbn = input("Enter ISBN: ")  # You can replace this with an input widget if needed
        book_instance = Book(isbn)
        book_instance.get_data()
        self.books.append(book_instance)
        self.update_list_widget()

    def remove_book(self, _):
        if self.list_widget.index >= 0 and self.list_widget.index < len(self.books):
            del self.books[self.list_widget.index]
            self.update_list_widget()

    def update_list_widget(self):
        self.list_widget.options = [f"{book.title} ({book.isbn})" for book in self.books]

# Example usage:
gallery_gui = GalleryGUI()

Enter ISBN: 